In [1]:
import json

coco_caption_path = "/home/why/data/caption/dataset_coco.json"


with open(coco_caption_path,'r') as j:
    captions = json.load(j)
    

# 数据处理

In [2]:
all_document = []


"""
数据结构

all_documents = [{ ‘name’: 图像名字， ‘document’: 图像的5句标注 } ,.....]

"""

for i in captions["images"]:
    d = {}
    d['name'] = i['filename']
    caps = []
    for j in i['sentences']:
        caps.append(j['raw'])
    caps = ' '.join(caps)
    d['document'] = caps
    all_document.append(d)

In [3]:
image_name = []
documents = []
for i in all_document:
    image_name.append(i['name'])
    documents.append(i['document'])

In [4]:
image_name[:10]

['COCO_val2014_000000391895.jpg',
 'COCO_val2014_000000522418.jpg',
 'COCO_val2014_000000184613.jpg',
 'COCO_val2014_000000318219.jpg',
 'COCO_val2014_000000554625.jpg',
 'COCO_val2014_000000397133.jpg',
 'COCO_val2014_000000574769.jpg',
 'COCO_val2014_000000060623.jpg',
 'COCO_val2014_000000309022.jpg',
 'COCO_val2014_000000005802.jpg']

In [5]:
documents[:10]

['A man with a red helmet on a small moped on a dirt road.  Man riding a motor bike on a dirt road on the countryside. A man riding on the back of a motorcycle. A dirt path with a young person on a motor bike rests to the foreground of a verdant area with a bridge and a background of cloud-wreathed mountains.  A man in a red shirt and a red hat is on a motorcycle on a hill side.',
 "A woman wearing a net on her head cutting a cake.  A woman cutting a large white sheet cake. A woman wearing a hair net cutting a large sheet cake. there is a woman that is cutting a white cake A woman marking a cake with the back of a chef's knife. ",
 'A child holding a flowered umbrella and petting a yak. A young man holding an umbrella next to a herd of cattle. a young boy barefoot holding an umbrella touching the horn of a cow A young boy with an umbrella who is touching the horn of a cow. A boy holding an umbrella while standing next to livestock.',
 'A young boy standing in front of a computer keyboa

# LDA

In [6]:
import json
from sklearn import feature_extraction
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
import lda

In [7]:
vectorizer = CountVectorizer(analyzer = 'word',stop_words = 'english', lowercase = True, min_df = 5)
X = vectorizer.fit_transform(documents)
word = vectorizer.get_feature_names()       # 词典

In [8]:
len(word)

9679

In [9]:
with open('data/all_word.json','w') as j:
    json.dump(word,j)

In [10]:
analyze = vectorizer.build_analyzer()
weight = X.toarray()

In [11]:
model = lda.LDA(n_topics = 60,n_iter = 500,random_state = 1)
model.fit(np.asarray(weight))

INFO:lda:n_documents: 123287
INFO:lda:vocab_size: 9679
INFO:lda:n_words: 3259779
INFO:lda:n_topics: 60
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -39755125
INFO:lda:<10> log likelihood: -22664960
INFO:lda:<20> log likelihood: -20345012
INFO:lda:<30> log likelihood: -19816190
INFO:lda:<40> log likelihood: -19626854
INFO:lda:<50> log likelihood: -19523755
INFO:lda:<60> log likelihood: -19455355
INFO:lda:<70> log likelihood: -19421524
INFO:lda:<80> log likelihood: -19396752
INFO:lda:<90> log likelihood: -19373498
INFO:lda:<100> log likelihood: -19353380
INFO:lda:<110> log likelihood: -19343395
INFO:lda:<120> log likelihood: -19330170
INFO:lda:<130> log likelihood: -19321965
INFO:lda:<140> log likelihood: -19313861
INFO:lda:<150> log likelihood: -19315659
INFO:lda:<160> log likelihood: -19307734
INFO:lda:<170> log likelihood: -19300044
INFO:lda:<180> log likelihood: -19300512
INFO:lda:<190> log likelihood: -19299390
INFO:lda:<200> log likelihood: -19297467
INFO:lda:<210> log likelih

In [12]:
'''每个主题的单词比重'''

topic_word = model.topic_word_
print(topic_word)

[[1.61041513e-07 1.61041513e-07 1.61041513e-07 ... 1.61041513e-07
  1.61041513e-07 1.61041513e-07]
 [1.62077733e-07 1.62077733e-07 1.62077733e-07 ... 1.62077733e-07
  1.62077733e-07 1.62077733e-07]
 [1.28207117e-07 1.28207117e-07 1.28207117e-07 ... 1.28207117e-07
  1.28207117e-07 1.28207117e-07]
 ...
 [1.75092129e-07 1.75092129e-07 1.75092129e-07 ... 1.75092129e-07
  1.75092129e-07 1.75092129e-07]
 [1.80437702e-07 1.80437702e-07 1.80437702e-07 ... 1.80437702e-07
  1.80437702e-07 1.80437702e-07]
 [2.15462493e-07 2.15462493e-07 2.15462493e-07 ... 2.15462493e-07
  4.33079612e-05 2.15462493e-07]]


In [13]:
len(topic_word)

60

In [14]:
len(topic_word[0])

9679

In [15]:
topic_word = topic_word.tolist()

In [16]:
with open('per_topic_worddiscrim.json','w') as j:
    json.dump(topic_word,j)

In [17]:
"""每个主题中最重要的20个单词"""
n = 20
N_top_word = []
for i, word_weight in enumerate(topic_word):
    distIndexArr = np.argsort(word_weight)
    topN_index = distIndexArr[:-(n+1):-1]
    topN_words = np.array(word)[topN_index]
    s = "*Topic "+str(i)+": "+' '.join(topN_words)
    N_top_word.append(s)
    #print('*Topic {}\n- {}'.format(i, ' '.join(topN_words)))

In [18]:
N_top_word

['*Topic 0: airplane plane flying sky large runway airport jet air blue small parked white tarmac sitting planes airplanes cloudy passenger landing',
 '*Topic 1: cat laying black sitting white sleeping lying couch floor cats ground orange resting chair brown shoes gray kitten lays sits',
 '*Topic 2: tennis court ball racket player man playing racquet holding hit swinging hitting match ready serve game male female rackets players',
 '*Topic 3: phone cell talking taking picture looking cellphone holding phones using person camera sitting smart ear photo takes talks selfie device',
 '*Topic 4: white black photo picture old photograph image close wedding vintage blurry bride couple groom shows background dark taken shot married',
 '*Topic 5: walking umbrella umbrellas holding street sidewalk people rain carrying walks walk person outside open near chairs past standing day city',
 '*Topic 6: playing game wii video remote room holding controller nintendo standing living control games couch c

In [2]:
with open('Topic_i_20_word.json','w') as j:
    json.dump(N_top_word,j)

NameError: name 'N_top_word' is not defined

In [19]:
"""获取每个主题的单词表示"""

word_20 = []

for i in N_top_word:
    i = i.split(': ')[1]
    word_20.append(i)

In [20]:
word_20

['airplane plane flying sky large runway airport jet air blue small parked white tarmac sitting planes airplanes cloudy passenger landing',
 'cat laying black sitting white sleeping lying couch floor cats ground orange resting chair brown shoes gray kitten lays sits',
 'tennis court ball racket player man playing racquet holding hit swinging hitting match ready serve game male female rackets players',
 'phone cell talking taking picture looking cellphone holding phones using person camera sitting smart ear photo takes talks selfie device',
 'white black photo picture old photograph image close wedding vintage blurry bride couple groom shows background dark taken shot married',
 'walking umbrella umbrellas holding street sidewalk people rain carrying walks walk person outside open near chairs past standing day city',
 'playing game wii video remote room holding controller nintendo standing living control games couch controllers people play television tv sitting',
 'baseball player bat b

In [21]:
with open('word_20.json','w') as j:
    json.dump(word_20,j)

In [22]:
'''判断文本的主题概率，为每个文本打上主题标签'''
doc_topic = model.doc_topic_
print("type(doc_topic):{}".format(type(doc_topic)))
print("shape:{}".format(doc_topic.shape))
label = []
for i in range(10):
    print(doc_topic[i])
    topic_most_pr = doc_topic[i].argmax()
    label.append(topic_most_pr)
    print("doc:{}topic:{}".format(i,topic_most_pr))
    
print(label)

type(doc_topic):<class 'numpy.ndarray'>
shape:(123287, 60)
[0.07045455 0.00227273 0.00227273 0.00227273 0.00227273 0.00227273
 0.00227273 0.00227273 0.00227273 0.00227273 0.00227273 0.00227273
 0.00227273 0.00227273 0.00227273 0.41136364 0.00227273 0.00227273
 0.04772727 0.00227273 0.00227273 0.025      0.00227273 0.00227273
 0.00227273 0.00227273 0.00227273 0.00227273 0.04772727 0.00227273
 0.00227273 0.00227273 0.00227273 0.00227273 0.025      0.00227273
 0.00227273 0.00227273 0.00227273 0.00227273 0.00227273 0.00227273
 0.00227273 0.00227273 0.11590909 0.00227273 0.00227273 0.00227273
 0.00227273 0.00227273 0.025      0.00227273 0.00227273 0.00227273
 0.00227273 0.09318182 0.00227273 0.025      0.00227273 0.00227273]
doc:0topic:15
[0.00285714 0.00285714 0.06       0.00285714 0.00285714 0.00285714
 0.00285714 0.00285714 0.00285714 0.00285714 0.00285714 0.00285714
 0.00285714 0.00285714 0.00285714 0.00285714 0.00285714 0.00285714
 0.00285714 0.00285714 0.00285714 0.08857143 0.40285714

In [23]:
doc_topic = doc_topic.tolist()
# with open('topic_distribution.json','w') as j:
#     json.dump(doc_topic,j)

In [24]:
len(doc_topic)

123287

In [25]:
len(doc_topic[0])

60

In [27]:
# import json

# with open('topic_distribution.json','r') as j:
#     doc_topic = json.load(j)


In [26]:
max_top = []
for i in doc_topic:
    m = i.index(max(i))
    max_top.append(m)


In [29]:
#max_top

In [30]:
dname = []

for i in image_name:
    i = i.split('_')[2]
    i = int(i.lstrip("0").split('.')[0])
    dname.append(i)

In [31]:
# dname

In [32]:
name_top = {}
name_top_word = {}

for n,i in enumerate(dname):
    
    name_top[i] = max_top[n]
    name_top_word[i] = word_20[max_top[n]]

In [33]:
"""数据结构：{image_name:top_1_topic}"""

# name_top

'数据结构：{image_name:top_1_topic}'

In [34]:
with open('top_1_topic_class.json','w') as j:
    json.dump(name_top,j)
    
with open('top_1_topic_word.json','w') as j:
    json.dump(name_top_word,j)

In [37]:
len(name_top_word)

123287

In [2]:
import os
import numpy as np
import h5py
import json
import torch
from tqdm import tqdm
from collections import Counter
from random import seed, choice, sample
import pickle

output_folder = "/home/why/image-captioning-bottom-up-top-down/other_dataset/"


with open('/home/why/data/caption/dataset_coco.json', 'r') as j:
    data = json.load(j)
    
with open("/home/why/image-captioning-bottom-up-top-down/other_dataset/WORDMAP_coco_5_cap_per_img_5_min_word_freq.json", 'r') as j:
    word_map = json.load(j)
    
with open("data/top_1_topic_class.json", 'r') as j:
    image_topclass= json.load(j)
    
with open("data/top_1_topic_word.json", 'r') as j:
    image_topword= json.load(j)



In [4]:
train_topic = []
val_topic = []
test_topic = []

train_topic_word = []
val_topic_word = []
test_topic_word = []
test_name = []


len_data = len(data['images'])
for num, img in enumerate(data['images']):
    
    image_id = img['filename'].split('_')[2]
    image_id = int(image_id.lstrip("0").split('.')[0])

    if img['split'] in {'train', 'restval'}:
        
        top_class = image_topclass[str(image_id)]
        word = image_topword[str(image_id)]
        word = word.split(' ')
        enc_w = [word_map.get(word, word_map['<unk>']) for word in word]
        
        train_topic.append(top_class)
        train_topic_word.append(enc_w)

    elif img['split'] in {'val'}:
        
        top_class = image_topclass[str(image_id)]
        word = image_topword[str(image_id)]
        word = word.split(' ')
        enc_w = [word_map.get(word, word_map['<unk>']) for word in word]
        
        val_topic.append(top_class)
        val_topic_word.append(enc_w)
  
    elif img['split'] in {'test'}:
        top_class = image_topclass[str(image_id)]
        word = image_topword[str(image_id)]
        word = word.split(' ')
        enc_w = [word_map.get(word, word_map['<unk>']) for word in word]
        
        
        test_name.append(image_id)
        test_topic.append(top_class)
        test_topic_word.append(enc_w)
 

In [5]:
with open('TEST_NAME.json', 'w') as j:
    json.dump(test_name, j)

In [40]:
with open('TRAIN_1_TOPIC_CLASS.json', 'w') as j:
    json.dump(train_topic, j)

with open('VAL_1_TOPIC_CLASS.json', 'w') as j:
    json.dump(val_topic, j)
    
with open('TEST_1_TOPIC_CLASS.json', 'w') as j:
    json.dump(test_topic, j)


with open('TRAIN_1_TOPIC_WORD.json', 'w') as j:
    json.dump(train_topic_word, j)

with open('VAL_1_TOPIC_WORD.json', 'w') as j:
    json.dump(val_topic_word, j)
    
with open('TEST_1_TOPIC_WORD.json', 'w') as j:
    json.dump(test_topic_word, j)


In [41]:
topic_class = []

topic_word = []



len_data = len(data['images'])
for num, img in enumerate(data['images']):
    
    image_id = img['filename'].split('_')[2]
    image_id = int(image_id.lstrip("0").split('.')[0])

    top_class = image_topclass[str(image_id)]
    word = image_topword[str(image_id)]
    word = word.split(' ')
    enc_w = [word_map.get(word, word_map['<unk>']) for word in word]

    topic_class.append(top_class)
    topic_word.append(enc_w)


In [43]:
with open('TOPIC_1_CLASS.json', 'w') as j:
    json.dump(topic_class, j)


with open('TOPIC_1_WORD.json', 'w') as j:
    json.dump(topic_word, j)
